<a href="https://colab.research.google.com/github/jsalbert/biotorch/blob/release/benchmark_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
!pip install biotorch --upgrade
!git clone https://github.com/jsalbert/biotorch.git

     |████████████████████████████████| 35.3 MB 59 kB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 100 kB 9.2 MB/s 
  Created wheel for biotorch: filename=biotorch-0.0.6-py3-none-any.whl size=78972 sha256=1ccd57e3e70493287133974eebc74b89098b82514fb581566f4dd9d659b1fd0e
  Stored in directory: /root/.cache/pip/wheels/bd/03/48/3f2559d4ef6135245075196550fe977448c0a7308cf561f764
Successfully built biotorch
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.5.4 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.
Cloning into 'biotorch'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 1

# Imports

In [2]:
import yaml
import torch
import torch.nn as nn
import torch.nn.functional as F

from biotorch.benchmark.run import Benchmark
from biotorch.module.biomodule import BioModule


# Create a PyTorch model and convert it to use Feedback Alignment

In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu3 = nn.ReLU()
        self.fc = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        out = self.pool1(self.relu1(self.conv1(x)))
        out = self.pool2(self.relu2(self.conv2(out)))
        out = self.relu3(self.conv3(out))
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        return self.fc(out)

In [4]:
# Create pytorch model
model = Model()
# Convert model to use Feedback Alignment `fa`
biomodel = BioModule(model, mode='fa')
# Print model
print(biomodel)
# Save model
torch.save(biomodel, 'biotorch/biomodel.pth')

Module has been converted to fa mode:

The layer configuration was:  {'type': 'fa', 'options': {'constrain_weights': False, 'gradient_clip': False, 'init': 'xavier'}}
- All the 3 <class 'torch.nn.modules.conv.Conv2d'> layers were converted successfully.
- All the 1 <class 'torch.nn.modules.linear.Linear'> layers were converted successfully.
BioModule(
  (module): Model(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (relu3): ReLU()
    (fc): Linear(in_features=128, out_features=10, bias=True)
  )
)


# Modify Configuration File to Append the Model Path

In [5]:
# Open an example configuration file
with open("biotorch/benchmark_configs/cifar10/le_net/sgd/fa.yaml", 'r') as stream:
    try:
        file = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Re write to put our custom model path
file['model'] = {
    'architecture': None,
    'loss_function': {'name': 'cross_entropy'},
    'mode': {'options': {'constrain_weights': False,
    'gradient_clip': False,
    'init': 'xavier'},
    'type': 'fa'},
    'pretrained': False,
    'checkpoint': 'biomodel.pth'
    }

file['experiment']['name'] = 'custom_model_fa'
# Deterministic won't work from Colab because we have to set CUDA environments
file['experiment']['deterministic'] = False

with open("biotorch/custom_model.yaml", 'w') as outfile:
    yaml.dump(file, outfile)  

# Run Benchmark

In [6]:
!ls

biotorch  sample_data


In [7]:
%cd biotorch

/content/biotorch


In [ ]:
benchmark = Benchmark('custom_model.yaml')
benchmark.run()

Preparing CIFAR-10 Benchmark Dataset and storing data in ./datasets/cifar10


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./datasets/cifar10/cifar-10-python.tar.gz to ./datasets/cifar10
Files already downloaded and verified
Files already downloaded and verified
Loading model checkpoint from  biomodel.pth
Adjusting learning rate of group 0 to 1.0000e-03.

Benchmarking model on CIFAR-10 Benchmark Dataset
{'display_iterations': 500, 'layer_alignment': True, 'top_k': 5, 'weight_ratio': True}


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][  0/351]	Time  0.224 ( 0.224)	Data  0.107 ( 0.107)	Loss 2.2935e+00 (2.2935e+00)	Acc@1  13.28 ( 13.28)	Acc@5  56.25 ( 56.25)
 * Acc@1 19.680 Acc@5 63.540
New best accuracy reached: 19.68000030517578 
Saving best accuracy model...
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch: [1][  0/351]	Time  0.065 ( 0.065)	Data  0.058 ( 0.058)	Loss 2.2218e+00 (2.2218e+00)	Acc@1  21.88 ( 21.88)	Acc@5  59.38 ( 59.38)
 * Acc@1 21.880 Acc@5 73.900
New best accuracy reached: 21.8799991607666 
Saving best accuracy model...
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch: [2][  0/351]	Time  0.071 ( 0.071)	Data  0.065 ( 0.065)	Loss 2.0950e+00 (2.0950e+00)	Acc@1  24.22 ( 24.22)	Acc@5  74.22 ( 74.22)
 * Acc@1 24.460 Acc@5 76.780
New best accuracy reached: 24.459999084472656 
Saving best accuracy model...
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch: [3][  0/351]	Time  0.061 ( 0.061)	Data  0.055 ( 0.055)	Loss 1.9832e+00 (1.9832e+00)	Acc@1  21.88 ( 21.88)	Acc@5  76.56 ( 76